<a href="https://colab.research.google.com/github/weedge/doraemon-nb/blob/main/megaTTS3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/bytedance/MegaTTS3


Cloning into 'MegaTTS3'...
remote: Enumerating objects: 178, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 178 (delta 33), reused 19 (delta 19), pack-reused 138 (from 2)
Receiving objects: 100% (178/178), 1.15 MiB | 14.70 MiB/s, done.
Resolving deltas: 100% (74/74), done.


In [2]:
%cd MegaTTS3

/content/MegaTTS3


In [ ]:
!pip install -q -r requirements.txt

In [ ]:
!pip3 install -U torch torchvision torchaudio

In [10]:
!pip3 show torch torchvision torchaudio

Name: torch
Version: 2.6.0
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3-Clause
Location: /usr/local/lib/python3.11/dist-packages
Requires: filelock, fsspec, jinja2, networkx, nvidia-cublas-cu12, nvidia-cuda-cupti-cu12, nvidia-cuda-nvrtc-cu12, nvidia-cuda-runtime-cu12, nvidia-cudnn-cu12, nvidia-cufft-cu12, nvidia-curand-cu12, nvidia-cusolver-cu12, nvidia-cusparse-cu12, nvidia-cusparselt-cu12, nvidia-nccl-cu12, nvidia-nvjitlink-cu12, nvidia-nvtx-cu12, sympy, triton, typing-extensions
Required-by: accelerate, fastai, openai-whisper, peft, sentence-transformers, timm, torchaudio, torchdiffeq, torchvision, x-transformers
---
Name: torchvision
Version: 0.21.0+cu124
Summary: image and video datasets and models for torch deep learning
Home-page: https://github.com/pytorch/vision
Author: PyTorch Core Team
Author-email: soumith@pytorch.org
License: BSD
Loc

# download

In [ ]:
!huggingface-cli download --quie ByteDance/MegaTTS3 --local-dir ./checkpoints --local-dir-use-symlinks False

# Inference

In [12]:
from IPython.display import Audio
Audio('assets/Chinese_prompt.wav',autoplay=True)

In [11]:
# p_w (intelligibility weight), t_w (similarity weight). Typically, prompt with more noises requires higher p_w and t_w
!PYTHONPATH="/content/MegaTTS3:$PYTHONPATH" \
  python tts/infer_cli.py --input_wav 'assets/Chinese_prompt.wav'  \
  --input_text "另一边的桌上,一位读书人嗤之以鼻道,'佛子三藏,神子燕小鱼是什么样的人物,李家的那个李子夜如何与他们相提并论？'" \
  --output_dir ./gen


| loaded 'dur_model' from './checkpoints/duration_lm/model_only_last.ckpt'.
| Missing keys: 0, Unexpected keys: 0
| loaded 'dit' from './checkpoints/diffusion_transformer/model_only_last.ckpt'.
| Missing keys: 0, Unexpected keys: 9
| loaded 'model' from './checkpoints/aligner_lm/model_only_last.ckpt'.
| Missing keys: 0, Unexpected keys: 0
2025-04-08 02:14:49.647821: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744078489.670247    4412 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744078489.675468    4412 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-08 02:14:49.692806: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow

In [13]:
from IPython.display import Audio
Audio("./gen/[P]另一边的桌上,一位读书人嗤之以鼻道,'佛.wav",autoplay=True)

In [14]:
from IPython.display import Audio
Audio('assets/English_prompt.wav',autoplay=True)

In [15]:
# As long as audio volume and pronunciation are appropriate, increasing --t_w within reasonable ranges (2.0~5.0)
# will increase the generated speech's expressiveness and similarity (especially for some emotional cases).
!PYTHONPATH="/content/MegaTTS3:$PYTHONPATH" \
  python tts/infer_cli.py --input_wav 'assets/English_prompt.wav' \
  --input_text 'As his long promised tariff threat turned into reality this week, top human advisers began fielding a wave of calls from business leaders, particularly in the automotive sector, along with lawmakers who were sounding the alarm.' \
  --output_dir ./gen --p_w 2.0 --t_w 3.0


| loaded 'dur_model' from './checkpoints/duration_lm/model_only_last.ckpt'.
| Missing keys: 0, Unexpected keys: 0
| loaded 'dit' from './checkpoints/diffusion_transformer/model_only_last.ckpt'.
| Missing keys: 0, Unexpected keys: 9
| loaded 'model' from './checkpoints/aligner_lm/model_only_last.ckpt'.
| Missing keys: 0, Unexpected keys: 0
2025-04-08 02:17:33.217327: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744078653.239342    5184 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744078653.244303    5184 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-08 02:17:33.261281: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow

In [16]:
from IPython.display import Audio
Audio('./gen/[P]As his long promised.wav',autoplay=True)

In [17]:
# When p_w (intelligibility weight) ≈ 1.0, the generated audio closely retains the speaker’s original accent. As p_w increases, it shifts toward standard pronunciation.
# t_w (similarity weight) is typically set 0–3 points higher than p_w for optimal results.
# Useful for accented TTS or solving the accent problems in cross-lingual TTS.
!PYTHONPATH="/content/MegaTTS3:$PYTHONPATH" \
  python tts/infer_cli.py --input_wav 'assets/English_prompt.wav' \
  --input_text '这是一条有口音的音频。' --output_dir ./gen --p_w 1.0 --t_w 3.0



| loaded 'dur_model' from './checkpoints/duration_lm/model_only_last.ckpt'.
| Missing keys: 0, Unexpected keys: 0
| loaded 'dit' from './checkpoints/diffusion_transformer/model_only_last.ckpt'.
| Missing keys: 0, Unexpected keys: 9
| loaded 'model' from './checkpoints/aligner_lm/model_only_last.ckpt'.
| Missing keys: 0, Unexpected keys: 0
2025-04-08 02:21:29.567519: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744078889.590305    6255 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744078889.595482    6255 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-08 02:21:29.612988: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow

In [18]:
from IPython.display import Audio
Audio('./gen/[P]这是一条有口音的音频。.wav',autoplay=True)

In [19]:
!PYTHONPATH="/content/MegaTTS3:$PYTHONPATH" \
  python tts/infer_cli.py --input_wav 'assets/English_prompt.wav' \
  --input_text '这条音频的发音标准一些了吗？' --output_dir ./gen --p_w 2.5 --t_w 2.5

| loaded 'dur_model' from './checkpoints/duration_lm/model_only_last.ckpt'.
| Missing keys: 0, Unexpected keys: 0
| loaded 'dit' from './checkpoints/diffusion_transformer/model_only_last.ckpt'.
| Missing keys: 0, Unexpected keys: 9
| loaded 'model' from './checkpoints/aligner_lm/model_only_last.ckpt'.
| Missing keys: 0, Unexpected keys: 0
2025-04-08 02:22:42.504017: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744078962.525999    6635 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744078962.531006    6635 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-08 02:22:42.548849: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow

In [20]:
from IPython.display import Audio
Audio('./gen/[P]这条音频的发音标准一些了吗？.wav',autoplay=True)